In [83]:
from langchain_ibm import ChatWatsonx
from langchain_openai import ChatOpenAI
from ibm_watsonx_ai.foundation_models import ModelInference
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from credentials import *
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import Literal, Annotated
import json
from pymongo import MongoClient
from typing import List, Dict

# llm = ChatWatsonx(
#     model_id="meta-llama/llama-3-3-70b-instruct",
#     url = ENDPOINT_URL,
# 	apikey = API_KEY,
# 	project_id = PROJECT_ID,
#     params = {
#         "decoding_method": "greedy",
#         "temperature": 0, 
#         "min_new_tokens": 5, 
#         "max_new_tokens": 2000
#     }
# )
llm = ChatOpenAI(
    model="gpt-4o",
    seed=420,
    temperature=0.01
)

llm.invoke([HumanMessage("Hello")])


APIConnectionError: Connection error.

In [14]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [36]:
query_translation_prompt = """
You are given a query in natural language and a json schema. The json schema is the schema of the json documents in a MongoDB collection that will be queried. You will translate the natural language query into a series of mongoDB queries. The queries will be executed with an "aggregate" function. You only have to return the mongodb queries, not the python code. since queries will be executed with an aggregate function, you can use any of the aggregate operators. Most of the times the query is not a simle filter but requires also a particolar view of the result (maybe pivots using the $unwind operator).

JSON SCHEMA:
{json_schema}

NATURAL LANGUAGE QUERY:
{query}
"""

query_translation_schema={
"title": "Query_Translation",
"description": "Translate a natural language query into a series of MongoDB queries that will be executed with an aggregate function",
  "type": "object",
  "properties": {
    "queries": {
      "description" : "A series of MongoDB queries that will be executed with an aggregate function",
      "type": "array",
      "items": {
        "type": "string",
        "description": "A MongoDB query, must be a key-value pair enclosed in curly braces."
      }
    }
  },
  "required": ["queries"],
  "additionalProperties": False
}

translation_llm = llm.with_structured_output(query_translation_schema)

with open('output/Menu/schema.json', 'r') as file:
    json_schema = file.read()

with open('output/Menu/description.txt', 'r') as file:
    collection_description = file.read()

collection_description_prompt = """
You are an agent that needs to answer queries from a set of documents. Here is a description of the document collection you will be working with:
{collection_description}
You need to use the tools that you are given to answer queries about the document collection. 
"""

system_prompt = collection_description_prompt.format(collection_description=collection_description)
# Connect to the local MongoDB server (default URI)
client = MongoClient("mongodb://localhost:27017/")

# Access a specific database (it will be created if it doesn't exist)
db = client["hackapizza"]

# Access a specific collection within the database
collection = db["Menu"]


In [37]:

for doc in collection.aggregate([
    {'$unwind': '$menu'}, {'$unwind': '$menu.ingredients'}, 
    {'$match': {'menu.ingredients.name': 'Amido di Stellarion'}}, 
    {'$project': {'_id': 0, 'dish_name': '$menu.dish_name'}}
]):
    print(doc)

In [41]:
translation_llm.invoke(query_translation_prompt.format(json_schema=json_schema, query="Quali piatti contengono Amido di Stellarion?"))

{'queries': [{'$unwind': '$restaurants'},
  {'$unwind': '$restaurants.menu'},
  {'$unwind': '$restaurants.menu.ingredients'},
  {'$match': {'restaurants.menu.ingredients.name': 'Amido di Stellarion'}},
  {'$project': {'_id': 0, 'dish_name': '$restaurants.menu.dish_name'}}]}

In [148]:

@tool
def translate_query_into_mongo(query : str)-> List[Dict[str,str]]:
    """function that takes a query in natural language and queries the mongodb"""
    prompt = query_translation_prompt.format(json_schema=json_schema, query=query)
    result = translation_llm.invoke(prompt)
    
    final_list = []

    for query in result["queries"]:
        try:
            if type(query) == str:
                final_list.append(eval(query))
            else:
                final_list.append(query)
        except Exception as e:
            print(f"Error: {query}")
            print(e)
            continue

    result = collection.aggregate(final_list)

# Itera sui risultati
    res = ""
    for doc in result:
        res += str(doc)
        print(doc)
    return res


@tool
def query_mongo(queries: List[Dict[str, str]])-> str:
    """function that takes a list of mongodb queries and executes the queries on the mongodb. returns the result of the queries"""
    return_string = ""
    print(f"queries: {type(queries[0])}")
    res = collection.aggregate(queries)
    for doc in res:
        return_string += str(doc)
        print(doc)
    return return_string



In [129]:
query = "Quali piatti contengono Amido di Stellarion?"
prompt = query_translation_prompt.format(json_schema=json_schema, query=query)
result = translation_llm.invoke(prompt)
result['queries']

['{ "$unwind": "$restaurants" }',
 '{ "$unwind": "$restaurants.menu" }',
 '{ "$unwind": "$restaurants.menu.ingredients" }',
 '{ "$match": { "restaurants.menu.ingredients.name": "Amido di Stellarion" } }',
 '{ "$project": { "dish_name": "$restaurants.menu.dish_name", "description": "$restaurants.menu.description", "price": "$restaurants.menu.price" } }']

In [131]:
eval(result['queries'][0])

{'$unwind': '$restaurants'}

In [150]:
agent = create_react_agent(llm, [translate_query_into_mongo], state_modifier=SystemMessage(system_prompt))

In [155]:
agent.invoke({'messages': HumanMessage('Quali piatti contengono Amido di Stellarion?')})

{'dish_name': 'Pizza Fra'}
{'dish_name': 'Sinfonia Tempolare Galattica'}
{'dish_name': 'Rinascita Cosmica'}
{'dish_name': "Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali"}
{'dish_name': 'Nebulosa di Drago Interdimensionale'}
{'dish_name': 'Cosmic Serenade'}
{'dish_name': 'Ecosistema Celeste'}
{'dish_name': 'Nebulosa Celestiale di Sogni Quantici'}
{'dish_name': 'Galassia Ardente'}
{'dish_name': 'Galassia di Cosmo-Delizie'}
{'dish_name': 'Nebulosa di Confini Sfondati'}
{'dish_name': 'Cosmos Risotto Reale'}


{'messages': [HumanMessage(content='Quali piatti contengono Amido di Stellarion?', additional_kwargs={}, response_metadata={}, id='02f417fc-5fa2-406b-9abe-dd22c4c567bb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gGKCGIC6dmy9b6p446uPcdGL', 'function': {'arguments': '{"query":"Quali piatti contengono Amido di Stellarion?"}', 'name': 'translate_query_into_mongo'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 855, 'total_tokens': 885, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-879614cc-fcf9-4a08-8a3a-dc8b17740131-0', tool_calls=[{'name': 'translate_query_into_mongo', 'args': {'query': 'Quali piatti c

In [6]:
class CreateSpecializedAgent:
    def __init__(self, llm, json_schema, collection_description, mongo_uri = "mongodb://localhost:27017/", db_name = "hackapizza", collection_name = "Documents_1"):
        self.llm = llm
        self.json_schema = json_schema
        self.collection_description = collection_description
        self.client = MongoClient(mongo_uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

        collection_description_prompt = """
        You are an agent that needs to answer queries from a set of documents. Here is a description of the document collection you will be working with:
        {collection_description}
        You need to use the tools that you are given to answer queries about the document collection. 
        """

        self.query_translation_prompt = """
        You are given a query in natural language and a json schema. The json schema is the schema of the json documents in a MongoDB collection that will be queried. You will translate the natural language query into a series of mongoDB queries. The queries will be executed with an "aggregate" function. You only have to return the mongodb queries, not the python code. since queries will be executed with an aggregate function, you can use any of the aggregate operators. Most of the times the query is not a simle filter but requires also a particolar view of the result (maybe pivots using the $unwind operator).

        JSON SCHEMA:
        {json_schema}

        NATURAL LANGUAGE QUERY:
        {query}
        """

        self.query_translation_schema={
        "title": "Query_Translation",
        "description": "Translate a natural language query into a series of MongoDB queries that will be executed with an aggregate function",
        "type": "object",
        "properties": {
            "queries": {
            "type": "array",
            "items": {
                "type": "string",
                "description": "A MongoDB query, must be a key-value pair enclosed in curly braces. both key and value must be strings enclosed in double quotes"
            }
            }
        },
        "required": ["queries"],
        "additionalProperties": False
        }

        collection_description_prompt = """
        You are an agent that needs to answer queries from a set of documents. Here is a description of the document collection you will be working with:
        {collection_description}
        You need to use the tools that you are given to answer queries about the document collection. 
        """
        
        self.translation_llm = llm.with_structured_output(self.query_translation_schema)
        self.system_prompt = collection_description_prompt.format(collection_description=collection_description)
        
        @tool
        def translate_query_into_mongo(query : Annotated[str,"the natural language query that will be translated in a list of mongodb queries to be executed"])-> List[dict]:
            """function that takes a query in natural language and returns a list of mongodb queries to execute. This tool is needed to query the document base to answer the user's query"""
            prompt = self.query_translation_prompt.format(json_schema=self.json_schema, query=query)
            result = self.translation_llm.invoke(prompt)
            queries = result["queries"]
            print(queries)
            return queries
        
        @tool
        def query_mongo(queries: Annotated[List[dict], "a list of dictionaries with key-value pair"])-> str:
            """function that takes a list of mongodb queries and returns the result of the queries"""
            return_string = ""
            res = self.collection.aggregate(queries)
            for doc in res:
                return_string += str(doc)
                print(doc)
            return return_string
    
        self.agent = create_react_agent(self.llm, [translate_query_into_mongo, query_mongo], state_modifier=SystemMessage(self.system_prompt))

        

        

In [7]:
agent = CreateSpecializedAgent(llm, json_schema, collection_description).agent

In [137]:
agent.invoke({'messages': HumanMessage('piatti che non hanno il basilico')})


{'dishes': [{'dish_name': 'Pizza Cri', 'signatureDish': True, 'description': 'Fusilli del Vento con Funghi dellʼEtere e Uova di Fenice.', 'ingredients': {'name': 'Fusilli del Vento', 'origin': 'Correnti aeree'}, 'techniques': [{'name': 'Affumicatura a Stratificazione Quantica', 'effect': 'Profumo cosmico'}, {'name': 'Sferificazione con Campi Magnetici Entropici', 'effect': 'Variazioni di texture'}, {'name': 'Microonde Entropiche Sincronizzate', 'effect': 'Cottura bilanciata'}], 'pairings': [{'beverage': 'Liquido Arcobaleno', 'notes': 'Esplosione di sapori e colori'}], 'price': '260 crediti galattici', 'notes': 'Un capolavoro multisensoriale.'}, {'dish_name': 'Pizza Cri', 'signatureDish': True, 'description': 'Fusilli del Vento con Funghi dellʼEtere e Uova di Fenice.', 'ingredients': {'name': 'Funghi dellʼEtere', 'origin': 'Nubi astrali'}, 'techniques': [{'name': 'Affumicatura a Stratificazione Quantica', 'effect': 'Profumo cosmico'}, {'name': 'Sferificazione con Campi Magnetici Entropi

{'messages': [HumanMessage(content='piatti che non hanno il basilico', additional_kwargs={}, response_metadata={}, id='27659bf2-726c-4ed2-af45-25f0a796ab7d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5hLhvuNYAHaU2pWHHc4cI52K', 'function': {'arguments': '{"query":"piatti che non hanno il basilico"}', 'name': 'translate_query_into_mongo'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 850, 'total_tokens': 875, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3e873b24-15a6-40ce-8b1a-0950b8d1d8fb-0', tool_calls=[{'name': 'translate_query_into_mongo', 'args': {'query': 'piatti che non hanno il basilico'}, 'i